<a href="https://colab.research.google.com/github/tnc-br/ddf_common/blob/vertex_ai_test/vertex_ai_poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This stub (ddfimport) allows the Ddf EE API to be imported.
import sys
!if [ ! -d "/content/ddf_common_stub" ] ; then git clone -b test https://github.com/tnc-br/ddf_common_stub.git; fi
sys.path.append("/content/ddf_common_stub/")
import ddfimport

# Use this line to import from a branch of the github repository.
# It will git clone the git repository under a google drive path.
# This allows you to modify the source files by opening the file view and
# changing files under /content/gdrive/MyDrive/<branch_name>
# ddfimport.ddf_source_control_pane()

# Alternatively, you can use this line to import from Main.
# If you import from Main, you will not be able to change files, but will not
# need a Google Login for Google Drive.
ddfimport.ddf_import_common()

executing checkout_branch ...
b''
main branch checked out as readonly. You may now use ddf_common imports


In [2]:
import importlib
import eeddf

# This may prompt for your Google account credentials to authenticate for
# Earth Engine. You may pass in an argument test_environment = True to access
# test data.
eeddf.initialize_ddf(test_environment = True)

In [3]:
%pip install --upgrade google-cloud-bigquery

In [18]:
from google.cloud import bigquery
_CLIENT = None
_TEST_CONFIG = {
    "TABLE" : "eval_results",
    "DATASET" : "harness_test_db",
    "PROJECT" : "river-sky-386919",
}

_PROD_CONFIG = {
    "TABLE" : "eval_results",
    "DATASET" : "harness_prod_db",
    "PROJECT" : "timberidprd",
}

def get_config():
  if eeddf.is_test_environment():
    global _TEST_CONFIG
    return _TEST_CONFIG
  global _PROD_CONFIG
  return _PROD_CONFIG

def get_big_query_client():
  global _CLIENT
  if not _CLIENT:
    _CLIENT = bigquery.Client(get_config()['PROJECT'])
  return _CLIENT

def read_eval_result(eval_id):
  client = get_big_query_client()

  # Set up SQL query
  table_name = f"{get_config()['DATASET']}.{get_config()['TABLE']}"
  query = f"SELECT * FROM {table_name} WHERE eval_id = '{eval_id}'"

  # Execute the query
  results = client.query_and_wait(query)
  return results

def insert_eval_result(eval, allow_overwrite=False):
  client = get_big_query_client()

  exists = read_eval_result(eval['eval_id']).total_rows
  if exists and not allow_overwrite:
    return [f"eval_id {eval['eval_id']} already exists"]

  # Set up reference to table we write to.
  table_ref = client.dataset(get_config()['DATASET']).table(get_config()['TABLE'])
  table = client.get_table(table_ref)

  # Automatically populate the timestamp field with the BigQuery commit time.
  eval['completion_timestamp'] = 'AUTO'

  # Insert data into BigQuery table.
  errors = client.insert_rows(table, [eval])
  return errors




In [16]:
import json

def test_write():
  eval_1 = {
    "eval_id" : 'd',
    "baseline_id" : "baseline_1",
    "experiment_id" : "fake_experiment_1",
    "fraction_fraudulent" : 0.5,
    "p_value_strategy" : "DEFAULT",
    "eval_pipeline_version_id" : "v1",
    "dataset_id" : "v1",
    "p_value_threshold_100km" : 0.03,
    "precision_100km": 0.7,
    "recall_100km" : 0.2,
    "auc_100km": 0.6,
    "p_value_threshold_300km" : 0.04,
    "precision_300km": 0.8,
    "recall_300km" : 0.4,
    "auc_300km": 0.7,
    "p_value_threshold_500km" : 0.03,
    "precision_500km": 0.7,
    "recall_500km" : 0.2,
    "auc_500km": 0.6,
  }

  eval_2 = {
    "eval_id" : 'e',
    "baseline_id" : "baseline_1",
    "experiment_id" : "fake_experiment_1",
    "fraction_fraudulent" : 0.6,
    "p_value_strategy" : "DEFAULT",
    "eval_pipeline_version_id" : "v1",
    "dataset_id" : "v1",
    "p_value_threshold_100km" : 0.03,
    "precision_100km": 0.8,
    "recall_100km" : 0.3,
    "auc_100km": 0.7,
    "p_value_threshold_300km" : 0.05,
    "precision_300km": 0.9,
    "recall_300km" : 0.5,
    "auc_300km": 0.8,
    "p_value_threshold_500km" : 0.05,
    "precision_500km": 0.8,
    "recall_500km" : 0.8,
    "auc_500km": 0.9,
  }

  eval_3 = {
    "eval_id" : 'f',
    "baseline_id" : "baseline_1",
    "experiment_id" : "fake_experiment_1",
    "fraction_fraudulent" : 0.5,
    "p_value_strategy" : "DEFAULT",
    "eval_pipeline_version_id" : "v1",
    "dataset_id" : "v1",
    "p_value_threshold_100km" : 0.3,
    "precision_100km": 0.9,
    "recall_100km" : 0.4,
    "auc_100km": 0.5,
    "p_value_threshold_300km" : 0.09,
    "precision_300km": 0.4,
    "recall_300km" : 0.3,
    "auc_300km": 0.2,
    "p_value_threshold_500km" : 0.04,
    "precision_500km": 0.3,
    "recall_500km" : 0.22,
    "auc_500km": 0.612,
  }

  errors = insert_eval_result(eval_1)
  assert len(errors) == 0

  errors = insert_eval_result(eval_2)
  assert len(errors) == 0

  errors = insert_eval_result(eval_3)
  assert len(errors) == 0

  result = read_eval_result('a')
  assert result.total_rows == 1

  rows = read_eval_result('b')
  assert result.total_rows == 1

  rows = read_eval_result('c')
  assert result.total_rows == 1


In [19]:
test_write()

[]
